In [1]:
#EDA for EAR Derivation

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

import irr
from multiprocessing import  Pool

from tqdm.auto import tqdm
tqdm.pandas()

warnings.filterwarnings('ignore')

source data data column definations

In [4]:
column_info = pd.read_csv('HomeCredit_columns_description.csv', encoding = 'unicode_escape')

In [5]:
column_info[column_info['Table'] == 'previous_application.csv'].head(5)

,Unnamed: 0,Table,Row,Description,Special
173,176,previous_application.csv,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
174,177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
175,178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",NaN
176,179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
177,180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the prev...,NaN


data loading

In [42]:
payments_file = 'installments_payments.csv'
previous_file = 'previous_application.csv'
df_payments = pd.read_csv(payments_file)
df_previous = pd.read_csv(previous_file)
(df_previous.shape, df_payments.shape)

((1670214, 37), (13605401, 8))

expected EAR

In [43]:
# select only required columns for previous applications and clean up unwanted dataframes
df_previous_filtered = df_previous[['SK_ID_CURR','SK_ID_PREV','AMT_ANNUITY','AMT_CREDIT','CNT_PAYMENT']]
#[df_previous['SK_ID_CURR'] == 100042]
df_previous_filtered.shape
del df_previous
gc.collect()

55

In [44]:
# handle null values
df_previous_filtered['AMT_ANNUITY'] = df_previous_filtered['AMT_ANNUITY'].fillna(0)
df_previous_filtered['AMT_CREDIT'] = df_previous_filtered['AMT_CREDIT'].fillna(0)
df_previous_filtered['CNT_PAYMENT'] = df_previous_filtered['CNT_PAYMENT'].fillna(0)

In [57]:
def get_ear(annuity, credit, no_of_payments):
    """ derives ear and returns
    """
    PMT = [-credit]
    PMT = PMT + ([annuity] * int(no_of_payments))
    d_irr = np.irr(PMT)
    
    return (1+d_irr)**12 - 1

In [58]:
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered\
.progress_apply(lambda x: get_ear(x.AMT_ANNUITY, x.AMT_CREDIT, x.CNT_PAYMENT), axis=1)

In [59]:
#df_previous_filtered[df_previous_filtered['EXPECTED_EAR'].isnull()]
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered['EXPECTED_EAR'].fillna(0)

In [60]:
# aggregate new column at the current application level.
df_expected_ear = df_previous_filtered.groupby('SK_ID_CURR')['EXPECTED_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_EXPECTED_EAR','median':'MEDIAN_EXPECTED_EAR', 'std':'STD_EXPECTED_EAR'})

In [61]:
df_expected_ear.head(6)

,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,
100001,1.199400,1.199400,NaN
100002,0.238453,0.238453,NaN
100003,0.368710,0.385640,0.083249
100004,0.360515,0.360515,NaN
100005,0.516463,0.516463,0.730389
100006,0.282858,0.342949,0.294920


In [62]:
df_expected_ear['MEAN_EXPECTED_EAR'] = df_expected_ear['MEAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['MEDIAN_EXPECTED_EAR'] = df_expected_ear['MEDIAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['STD_EXPECTED_EAR'] = df_expected_ear['STD_EXPECTED_EAR'].fillna(0)

In [63]:
df_expected_ear.head(6)

,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,
100001,1.199400,1.199400,0.000000
100002,0.238453,0.238453,0.000000
100003,0.368710,0.385640,0.083249
100004,0.360515,0.360515,0.000000
100005,0.516463,0.516463,0.730389
100006,0.282858,0.342949,0.294920


Actual EAR

In [64]:
# select required installments 
df_payments_filtered = df_payments[['SK_ID_PREV','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT']][df_payments['SK_ID_PREV'].isin(df_previous_filtered['SK_ID_PREV'].tolist())]

In [65]:
df_payments_filtered.shape

(12354575, 4)

In [66]:
# merge with previous application to get AMT_CREDIT
df_previous_payments = df_payments_filtered\
.merge(df_previous_filtered[['SK_ID_CURR','SK_ID_PREV','EXPECTED_EAR','AMT_CREDIT']], on='SK_ID_PREV', how='inner')

In [67]:
df_previous_payments.shape

(12354575, 7)

In [68]:
# handle nulls
df_previous_payments['AMT_CREDIT'] = df_previous_payments['AMT_CREDIT'].fillna(0)
df_previous_payments['DAYS_INSTALMENT'] = df_previous_payments['DAYS_INSTALMENT'].fillna(0)
df_previous_payments['DAYS_ENTRY_PAYMENT'] = df_previous_payments['DAYS_ENTRY_PAYMENT'].fillna(0)
df_previous_payments['AMT_INSTALMENT'] = df_previous_payments['AMT_INSTALMENT'].fillna(0)

In [69]:
# derive start date
df_previous_payments = df_previous_payments\
.join((df_previous_payments.groupby('SK_ID_PREV')['DAYS_INSTALMENT']\
.agg(['min'])-30).rename(columns={'min':'START_DATE'}), on='SK_ID_PREV')

In [70]:
# derive end date
df_previous_payments = df_previous_payments\
.join(df_previous_payments.groupby('SK_ID_PREV')['DAYS_ENTRY_PAYMENT']\
.agg(['max']).rename(columns={'max':'END_DATE'}), on='SK_ID_PREV')

In [71]:
# index column based on installments to create installaments over time.
df_previous_payments['INDEX'] = df_previous_payments['DAYS_ENTRY_PAYMENT'] - df_previous_payments['START_DATE']

In [72]:
#df_previous_payments[df_previous_payments['SK_ID_PREV'] == 2604725]

In [73]:
def get_dict(df):
    keys = df['INDEX'].tolist()
    values = df['AMT_INSTALMENT'].tolist()
    dictionary = dict(zip(keys, values))
    
    return dictionary

In [74]:
# first prepare the dictionary with index and amount paid at that particular interval
df_previous_payments_t = df_previous_payments.groupby(['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE', 'AMT_CREDIT'])\
.progress_apply(lambda x: get_dict(x))

In [75]:
df_previous_payments_t = df_previous_payments_t.reset_index()
df_previous_payments_t.columns = ['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE','AMT_CREDIT', 'TMP_EAR_DICT']

In [76]:
df_previous_payments_t.head(5)

,SK_ID_CURR,SK_ID_PREV,START_DATE,END_DATE,AMT_CREDIT,TMP_EAR_DICT
0,100001,1369693,-1739.0,-1628.0,23787.0,"{24.0: 3951.0, 111.0: 17397.9, 79.0: 3951.0}"
1,100002,1038818,-595.0,-49.0,179055.0,"{462.0: 9251.775, 220.0: 9251.775, 546.0: 5309..."
2,100003,1810518,-746.0,-544.0,1035882.0,"{56.0: 98356.995, 85.0: 98356.995, 202.0: 5608..."
3,100003,2396755,-2340.0,-1985.0,68053.5,"{114.0: 6737.31, 269.0: 6737.31, 141.0: 6737.3..."
4,100003,2636178,-827.0,-661.0,348637.5,"{137.0: 64567.665, 21.0: 64567.665, 108.0: 645..."


In [77]:
# there are a few cases where the length of the cashflow does not match with installments.
def get_cf(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT):
    credit = AMT_CREDIT
    period_length = int(END_DATE - START_DATE)
    d = TMP_EAR_DICT
    if period_length > 0:
        cf = [0] * (period_length + 1)
        cf[0] = -credit
        for index, amount in d.items():
            try:
                cf[int(index)] = amount
            except Exception as e:
                print(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT)
    else:
        return [0]
    return cf

In [78]:
df_previous_payments_tt = df_previous_payments_t\
.progress_apply(lambda x: get_cf(x.SK_ID_CURR, x.SK_ID_PREV, x.START_DATE, x.END_DATE, x.AMT_CREDIT, x.TMP_EAR_DICT), axis=1)

108966 2805403 -170.0 -159.0 47290.5 {-20.0: 8744.94, 11.0: 33455.655, 0.0: 8744.94}
123458 2755582 -1838.0 -1787.0 30195.0 {-274.0: 5652.495, 0.0: 5652.495, 51.0: 16403.67, -239.0: 5652.495}
123458 2755582 -1838.0 -1787.0 30195.0 {-274.0: 5652.495, 0.0: 5652.495, 51.0: 16403.67, -239.0: 5652.495}
125014 1181029 -1177.0 -1163.0 32184.0 {0.0: 3858.84, -22.0: 3858.84, 14.0: 27725.355}
152554 1377212 -444.0 -443.0 17347.5 {-20.0: 3567.015, 1.0: 15827.58}
163044 2346107 -617.0 -572.0 148365.0 {45.0: 144025.785, -67.0: 10704.555}
171336 2049507 -938.0 -881.0 24682.5 {57.0: 4361.4, 27.0: 4361.4, -98.0: 4361.4}
174087 1214702 -1284.0 -1268.0 18351.0 {0.0: 3993.39, -9.0: 3993.39, -23.0: 3993.39, 16.0: 3990.105}
188263 2617776 -1220.0 -1209.0 26284.5 {11.0: 22766.985, -20.0: 3196.215, 0.0: 3196.215}
192645 1850683 -820.0 -798.0 15246.0 {22.0: 7864.515, -46.0: 3964.095, 0.0: 3964.095}
195834 2044431 -719.0 -634.0 67828.5 {-177.0: 12285.36, -173.0: 12285.36, 85.0: 47251.44}
195834 2044431 -719.0 

In [79]:
# convert to datafame and join to original dataset using index column
df_previous_payments_tt = df_previous_payments_tt.to_frame()
df_previous_payments_tt.columns = ['CF']
df_previous_payments_tt_j = df_previous_payments_t.join(df_previous_payments_tt, on=None, how='inner')

In [80]:
df_previous_payments_tt_j.shape

(958905, 7)

In [98]:
def get_actual_ear(cf):
    """ derives actual EAR based on all payments based on previous application
    """
    n_irr = irr.irr(cf)
    try:
        ear = (1+n_irr)**360 - 1 # we take 1 year to be 360 days
    except Exception as e:
        ear = 0
    return ear

In [99]:
# derive ear based on cashflow column (took 12 hours)
# df_actual_ear = df_previous_payments_tt_j.progress_apply(lambda x: get_actual_ear(x.CF), axis=1)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [101]:
df_final = df_actual_ear.to_frame()
df_final.columns = ['ACTUAL_EAR']
df_final.shape

(958905, 1)

In [102]:
# join results to main dataset using index column
df_final_j = df_previous_payments_tt_j[['SK_ID_CURR','SK_ID_PREV']]\
.join(df_final, on=None, how='inner')

In [104]:
df_final_j.shape

(958905, 3)

In [105]:
# aggregate ear at current application level
df_final_j = df_final_j.groupby('SK_ID_CURR')['ACTUAL_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_ACTUAL_EAR','median':'MEDIAN_ACTUAL_EAR', 'std':'STD_ACTUAL_EAR'})

In [107]:
df_final_j['STD_ACTUAL_EAR'] = df_final_j['STD_ACTUAL_EAR'].fillna(0)

In [109]:
df_final_j.shape

(336935, 3)

In [110]:
# joining expected and actual ear
df_expected_actual = df_final_j.join(df_expected_ear, on='SK_ID_CURR', how='inner')

In [114]:
df_expected_actual.shape

(336935, 6)

In [115]:
# writing both actual and expected ear
actual_expected_ear_csv = '/ws_nparigi/01_Work/04_mba/Year2/Block1/valuation/datascience-assignment/home-credit-default-risk/actual_expected_ear.csv'
df_expected_actual.to_csv(actual_expected_ear_csv, index=True)

In [116]:
# writing only expected ear
expected_ear_csv = '/ws_nparigi/01_Work/04_mba/Year2/Block1/valuation/datascience-assignment/home-credit-default-risk/expected_ear.csv'
df_expected_ear.to_csv(expected_ear_csv, index=True)


In [117]:
# writing only actual ear
actual_ear_csv = '/ws_nparigi/01_Work/04_mba/Year2/Block1/valuation/datascience-assignment/home-credit-default-risk/actual_ear.csv'
df_final_j.to_csv(actual_ear_csv, index=True)
